# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


* Derivation of gradient of softmax loss   
Let $\mathbf{x}$ be an input vector of shape (1, D), $\mathbf{W}$ be the weight matrix of shape (D, C). $C$ is the number of classification, cross entropy function is:
$$
	CE(\mathbf{y},\hat{\mathbf{y}}) = -\sum_i \mathbf{y}_i \log{\hat{\mathbf{y}}_i}
$$
where $\mathbf{y}$ is the one-hot label vector, and $\hat{\mathbf{y}} = \text{softmax}(\mathbf{s}), \mathbf{s} = \mathbf{x}\mathbf{W}$ is the predicted probability vector for all classes. 
Assume $\mathbf{x}$ is of class k, i.e. $\mathbf{y}_k = 1$, then loss function becomes
$$  
L =CE(\mathbf{y},\hat{\mathbf{y}})= -\log(\frac{e^{\mathbf{s}_k}}{\sum_j e^{\mathbf{s}_j} })
$$

* derivation of softmax function   
	denote $\mathbf{h} = \hat{\mathbf{y}} = \text{softmax}(\mathbf{s})$, $\mathbf{h} = (h_1,...,h_i,...,h_C) $
	\begin{align*}
		f_i & = e^{s_i}\\
		g_i & = \sum_i e^{s_i}\\
		h_i & = \frac{f_i}{g_i} \\
	\end{align*}
	then there is 
	$$
		\frac{\partial h_i}{\partial \mathbf{s}_j} = 
			\begin{cases}
			h_i(1-h_i) & i=j\\
			-h_ih_j & i\ne j
			\end{cases}
	$$
	As $L = CE(\mathbf{y},\hat{\mathbf{y}}) = -\log(h_k)$, then
	\begin{align*}
		\frac{\partial L}{\partial \mathbf{s}_j} 
				& = -\frac{1}{h_k} \frac{\partial h_k}{\partial \mathbf{s}_j}\\
				& = \begin{cases}
						h_k - 1 & j = k\\
						h_j     & j \ne k
					\end{cases}
	\end{align*}
	thus
	$$
		\frac{\partial L}{\partial \mathbf{s}} = (h_1,h_2,...,h_k-1,...,h_C) = \mathbf{h} - \mathbf{y}
	$$
* derivation of weight matrix   
	\begin{align*}
		\frac{\partial L}{\partial \mathbf{W}_{ij}} 
			& = \sum_t \frac{\partial L}{\partial \mathbf{s}_t} \frac{\partial \mathbf{s}_t}{\partial \mathbf{W}_{ij}} \\
			& = \frac{\partial L}{\partial \mathbf{s}_j} \mathbf{x}_i 
	\end{align*}
	then
	\begin{align*}
		\frac{\partial L}{\partial \mathbf{W}}
			& = \mathbf{x} \frac{\partial L}{\partial \mathbf{s}}\\
			& = \mathbf{x}(\mathbf{h} - \mathbf{y})
	\end{align*}
* When $\mathbf{X}$ is the batch input matrix   
	Then $H = softmax(S), S = XW$, and $H,S$ both has N rows corresponding to N samples, denote them as $H_i, W_i$
	$$
		\frac{\partial L}{\partial \mathbf{W}} =
		\sum_{i=1}^N \frac{\partial L(S_i)}{\partial \mathbf{W}} 
	$$
	Here $L(S_i)$ is the loss of ith sample.

	The result is similar.
	$$
		\frac{\partial L}{\partial \mathbf{W}} = \mathbf{X}(\mathbf{H}-\mathbf{y})
	$$
	

In [3]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.343357
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *When initiate, W is very close to zero matrix, then elements of s = XW is approximate to 0, as there are totally 10 classes, so $softmax(s_k) \approx 1/10 = 0.1$*


In [4]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 1.610124 analytic: 1.610124, relative error: 2.944681e-08
numerical: 2.737077 analytic: 2.737076, relative error: 1.651850e-08
numerical: -1.635681 analytic: -1.635681, relative error: 5.704797e-09
numerical: 0.684445 analytic: 0.684445, relative error: 7.244286e-08
numerical: -1.198539 analytic: -1.198539, relative error: 5.919285e-08
numerical: 0.785467 analytic: 0.785467, relative error: 3.656680e-08
numerical: -2.586871 analytic: -2.586872, relative error: 1.555177e-08
numerical: 0.004238 analytic: 0.004238, relative error: 5.321374e-06
numerical: -0.211423 analytic: -0.211423, relative error: 5.882414e-08
numerical: 1.078812 analytic: 1.078812, relative error: 3.464727e-08
numerical: -0.594388 analytic: -0.594388, relative error: 1.123393e-07
numerical: 1.848497 analytic: 1.848497, relative error: 3.131213e-09
numerical: 1.048987 analytic: 1.048986, relative error: 5.537070e-08
numerical: 0.784118 analytic: 0.784118, relative error: 1.480404e-08
numerical: -2.824823 ana

In [5]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.343357e+00 computed in 0.066943s
vectorized loss: 2.343357e+00 computed in 0.000000s
Loss difference: 0.000000
Gradient difference: 0.000000


In [6]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [4e-8, 5e-7]
regularization_strengths = [5e2, 2e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
# Your code

def acc_eval(X_train, y_train, X_val, y_val, model, verbose=False):
    y_train_pred = model.predict(X_train)
    acc_train = np.mean(y_train == y_train_pred)
    y_val_pred = model.predict(X_val)
    acc_val = np.mean(y_val == y_val_pred)
    if verbose:
        print('training accuracy: %f, validation accuracy: %f' % (acc_train, acc_val ))
    return acc_train, acc_val

# x represents learning_rate, y represents regularization strength
nx = 6; ny = 30
x = np.linspace(learning_rates[0], learning_rates[1], num=nx)
y = np.linspace(regularization_strengths[0], regularization_strengths[1], num=ny)
xv, yv = np.meshgrid(x,y)
points = np.c_[xv.ravel(), yv.ravel()]
for xy in points:
    model = Softmax()
    lr = xy[0]; reg = xy[1]
    print('learning rate = %2.2e, regularization strength = %2.2e' %(lr, reg))
    model.train(X_train, y_train, learning_rate=lr, reg=reg,
                      num_iters=4500)
    acc_train, acc_val = acc_eval(X_train, y_train, X_val, y_val, model, verbose=True)
    results[(lr,reg)] = (acc_train, acc_val)
    if best_val <= acc_val:
        best_val = acc_val
        best_softmax = model



################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

learning rate = 5.00e-08, regularization strength = 1.00e+03
training accuracy: 0.296776
validation accuracy: 0.314000
learning rate = 1.62e-07, regularization strength = 1.00e+03
training accuracy: 0.388102
validation accuracy: 0.391000
learning rate = 2.75e-07, regularization strength = 1.00e+03
training accuracy: 0.405653
validation accuracy: 0.413000
learning rate = 3.87e-07, regularization strength = 1.00e+03
training accuracy: 0.403429
validation accuracy: 0.404000
learning rate = 5.00e-07, regularization strength = 1.00e+03
training accuracy: 0.407082
validation accuracy: 0.406000
learning rate = 5.00e-08, regularization strength = 3.58e+03
training accuracy: 0.352469
validation accuracy: 0.370000
learning rate = 1.62e-07, regularization strength = 3.58e+03
training accuracy: 0.384245
validation accuracy: 0.396000
learning rate = 2.75e-07, regularization strength = 3.58e+03
training accuracy: 0.385082
validation accuracy: 0.388000
learning rate = 3.87e-07, regularization strengt

training accuracy: 0.319816
validation accuracy: 0.336000
learning rate = 5.00e-08, regularization strength = 3.71e+04
training accuracy: 0.317020
validation accuracy: 0.331000
learning rate = 1.62e-07, regularization strength = 3.71e+04
training accuracy: 0.311347
validation accuracy: 0.327000
learning rate = 2.75e-07, regularization strength = 3.71e+04
training accuracy: 0.313857
validation accuracy: 0.321000
learning rate = 3.87e-07, regularization strength = 3.71e+04
training accuracy: 0.304286
validation accuracy: 0.317000
learning rate = 5.00e-07, regularization strength = 3.71e+04
training accuracy: 0.302041
validation accuracy: 0.322000
learning rate = 5.00e-08, regularization strength = 3.97e+04
training accuracy: 0.308204
validation accuracy: 0.330000
learning rate = 1.62e-07, regularization strength = 3.97e+04
training accuracy: 0.303694
validation accuracy: 0.322000
learning rate = 2.75e-07, regularization strength = 3.97e+04
training accuracy: 0.304184
validation accuracy:

KeyboardInterrupt: 

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question** - *True or False*

It's possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*: True

*Your explanation*: When the score of the new datapoint satisfies that $ s_{y_i} - s_j -\Delta >0$, then SVM loss does not change, but the Softmax classifier loss will change.

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])